In [1]:
import pandas as pd
import teradatasql

In [2]:
#Lee el archivo de los proveedores
def leer_archivos():
    url = ""
    nombre_prove = "Provedores.xlsx"
    nombre_historico = "Historico.xlsx"
    
    url_prove = url + nombre_provee
    url_his = url + nombre_historico
    
    dtype = {
        '' : str,
    
    }

    df_prove = pd.read_excel(url_prove, dtype=dtype)
    df_historico = pd.read_excel(ulr_historico, dtype = dtype)
    
    return df_prove, df_historico

def quitar_duplicados_e_historicos(df_prove,df_historico):

    #Crea la llave
    df_prove["Llave"] = df_prove["Cuenta"] + df_prove["Aut"] + df_prove["Monto"] + df_prove["Fecha"]

    #Borramos los duplicados
    df_prove = df_prove.drop_duplicates(subset = 'Llave')

    #Eliminamos las que ya se encuentrann en el historico
    df_prove = df_prove[~df_prove['Llave'].isin(set(df_historico['Llave']))]
    
    return df_prove
    
def teradata_arsd(cuentas):
    teradata = f"""SELECT NUM_AUT_COM AS Compra,  NUM_CTA_CTAF AS Cuenta
    , (VAL_TRANS_COM - VAL_JUROS_PARC_COM) AS Monto
    , DAT_TRANS_COM AS fechacompra
    ,CASE WHEN VAL_DADO_ORIG_LKP IN ('320001','350001','353001','356001','310001','311001','373001','376001','372001','352001') THEN '1000'
    WHEN  VAL_DADO_ORIG_LKP  IN ('320002','350002','353002','356002','310002','311002','373002','376002','352002','372002','377002','357002','354002','374002') THEN '1100'
    WHEN  VAL_DADO_ORIG_LKP IN ('320003','350003','353003','356003','310003','311003','373003','376003','352003','372003','377003','357003','354003','374003') THEN '1181'
    END AS PLAN
    , NOM_ESTB_PRO
    ,'D002' AS Descripcion
    ,COD_CICL_FAT_FCCFM AS corte
    ,ID_PROD_CTAF
    ,Val_dado_orig_can 
    ,NUM_REF_COM as Reference 
    FROM COMPRA_V 
    INNER JOIN CARTAO_FINANCIAMENTO_V 
    ON ID_CAR_COM = ID_CAR_CARF 
    INNER JOIN CONTA_FINANCIAMENTO_V 
    ON ID_CTA_CTAF = ID_CTA_CARF 
    LEFT JOIN CANAL_V 
    ON ID_CANA_CAN=ID_CANA_ADES_CTAF 
    INNER JOIN TIPO_TRANSACAO_V 
    ON ID_TIPO_TRANS_LKP = ID_TIPO_TRANS_COM 
    INNER JOIN FAT_COMP_CONTA_FINANC_FDM_V 
    ON NUM_CTA_CTAF = NUM_CTA_ATUAL_FCCFM 
    AND ID_CICL_ATRA_ROLLRATE_FCCFM IN ('11','12') 
    AND VAL_SALDO_DIV_TOT_FCCFM > 0 
         INNER JOIN TRANSACAO_PROSA_V 
    ON ID_CAR_PRO=ID_CAR_COM 
    AND NUM_AUT_COM = NUM_AUT_PRO 
    AND NUM_REF_COM = NUM_REF_PRO 
    WHERE 
    ID_PROD_CTAF IN (9, 38, 24, 52, 11, 14, 19, 30, 36, 47,35,37,57,40)
    AND ((COD_CICL_FAT_FCCFM=10 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >10 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+10, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+10,-1)  
     END AS z) AND CURRENT_DATE) 
     OR (COD_CICL_FAT_FCCFM=25 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >25 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+25, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+25,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=3 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >3 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+3, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+3,-1)  
     END AS z) AND CURRENT_DATE)
                     OR (COD_CICL_FAT_FCCFM=5 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >5 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+5, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+5,-1)  
     END AS z) AND CURRENT_DATE)
       OR (COD_CICL_FAT_FCCFM=15 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >15 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+15, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+15,-1)  
     END AS z) AND CURRENT_DATE)
        OR (COD_CICL_FAT_FCCFM=20 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >20 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+20, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+20,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=11 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >11 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+11, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+11,-1)  
     END AS z) AND CURRENT_DATE)
       OR (COD_CICL_FAT_FCCFM=28 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >28 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+28, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+28,-1)  
     END AS z) AND CURRENT_DATE)
       OR (COD_CICL_FAT_FCCFM=23 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >23 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+23, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+23,-1)  
     END AS z) AND CURRENT_DATE)
        OR (COD_CICL_FAT_FCCFM=8 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >8 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+8, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+8,-1)  
     END AS z) AND CURRENT_DATE)
     )AND ID_MES_FCCFM = ADD_MONTHS(CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) +1, -1)
    AND VAL_DADO_ORIG_LKP IN ('320002', '320003', '320001','350002', '350003', '350001','353002', '353003', '353001','310002', '310003', '310001','311002', '311003', '311001','373002', '373003', '373001','376002', '376003', '376001','356002', '356003', '356001','352001','372001','352002','372002','352003','372003','377002','357002','354002','374002','377003','357003','354003','374003')
    AND NUM_CTA_CTAF IN ({cuentas})
    ORDER BY NUM_CTA_CTAF, Monto DESC
    """
    
    return teradata
    
def teradata_artd(cuentas):
    teradata = f"""SELECT NUM_AUT_COM AS Compra,  NUM_CTA_CTAF AS Cuenta
    , (VAL_TRANS_COM - VAL_JUROS_PARC_COM) AS Monto
    , DAT_TRANS_COM AS fechacompra
    ,CASE WHEN VAL_DADO_ORIG_LKP IN ('320001') THEN '1000'
    WHEN VAL_DADO_ORIG_LKP in ('311001','310001') THEN '6000'
    WHEN  VAL_DADO_ORIG_LKP  IN ('320002','350002','353002','356002','310002','311002','373002','376002','352002','372002','377002','357002','354002','374002') THEN '1100'
    WHEN  VAL_DADO_ORIG_LKP IN ('320003','350003','353003','356003','310003','311003','373003','376003','352003','372003','377003','357003','354003','374003') THEN '1181'
    END AS PLAN
    , NOM_ESTB_PRO
    ,'D002' AS Descripcion
    ,COD_CICL_FAT_FCCFM AS corte

    ,ID_PROD_CTAF

    ,Val_dado_orig_can 
    ,NUM_REF_COM as Reference 
    FROM COMPRA_V 
    INNER JOIN CARTAO_FINANCIAMENTO_V 
    ON ID_CAR_COM = ID_CAR_CARF 
    INNER JOIN CONTA_FINANCIAMENTO_V 
    ON ID_CTA_CTAF = ID_CTA_CARF 
    LEFT JOIN CANAL_V 
    ON ID_CANA_CAN=ID_CANA_ADES_CTAF 
    INNER JOIN TIPO_TRANSACAO_V 
    ON ID_TIPO_TRANS_LKP = ID_TIPO_TRANS_COM 
    INNER JOIN FAT_COMP_CONTA_FINANC_FDM_V 
    ON NUM_CTA_CTAF = NUM_CTA_ATUAL_FCCFM 
    AND ID_CICL_ATRA_ROLLRATE_FCCFM IN ('11','12') 
    AND VAL_SALDO_DIV_TOT_FCCFM > 0 
         full OUTER JOIN TRANSACAO_PROSA_V 
    ON ID_CAR_PRO=ID_CAR_COM 
    AND NUM_AUT_COM = NUM_AUT_PRO 
    AND NUM_REF_COM = NUM_REF_PRO 
    WHERE 
    ID_PROD_CTAF IN (9, 38, 24, 52, 11, 14, 19, 30, 36, 47,35,37,57,40)
    AND ((COD_CICL_FAT_FCCFM=10 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >10 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+10, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+10,-1)  
     END AS z) AND CURRENT_DATE) 
     OR (COD_CICL_FAT_FCCFM=25 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >25 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+25, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+25,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=3 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >3 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+3, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+3,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=5 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >5 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+5, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+5,-1)  
     END AS z) AND CURRENT_DATE)
       OR (COD_CICL_FAT_FCCFM=15 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >15 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+15, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+15,-1)  
     END AS z) AND CURRENT_DATE)
        OR (COD_CICL_FAT_FCCFM=20 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >20 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+20, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+20,-1)  
     END AS z) AND CURRENT_DATE)
         OR (COD_CICL_FAT_FCCFM=28 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >28 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+28, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+28,-1)  
     END AS z) AND CURRENT_DATE)
         OR (COD_CICL_FAT_FCCFM=23 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >23 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+23, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+23,-1)  
     END AS z) AND CURRENT_DATE)
         OR (COD_CICL_FAT_FCCFM=8 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >8 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+8, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+8,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=11 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >11 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+11, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+11,-1)  
     END AS z) AND CURRENT_DATE)
     )AND (ID_MES_FCCFM = ADD_MONTHS(CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) +1, -1)
     or ID_MES_FCCFM = ADD_MONTHS(CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) +1, 0))
    AND VAL_DADO_ORIG_LKP IN ('320002', '320003', '320001','350002', '350003', '350001','353002', '353003', '353001','310002', '310003', '310001','311002', '311003', '311001','373002', '373003', '373001','376002', '376003', '376001','356002', '356003', '356001','352001','372001','352002','372002','352003','372003','377002','357002','354002','374002','377003','357003','354003','374003')
    AND NUM_CTA_CTAF IN ({cuentas})
    ORDER BY NUM_CTA_CTAF, Monto DESC
    
    """
    
    return teradata

def conexion_a_teradata():
    conn = teradatasql.connect(
        host='',
        user='',
        password=''
    )
    
def hacer_consulta(query):
    df = pd.read_sql(query, conn)

def cerrar_conexion(query):
    conn.close()
    
conexion_a_teradata()



OperationalError: A hostname or IP address must be specified for the host connection parameter
 at gosqldriver/teradatasql.formatError ErrorUtil.go:85
 at gosqldriver/teradatasql.ParseConParams ConParams.go:195
 at main.parseParams goside.go:218
 at main.goParseParams goside.go:210
 at _cgoexp_ff5e33a08e40_goParseParams _cgo_gotypes.go:231
 at runtime.cgocallbackg1 cgocall.go:446
 at runtime.cgocallbackg cgocall.go:350
 at runtime.cgocallback asm_amd64.s:1084
 at runtime.goexit asm_amd64.s:1700